In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import lightgbm as lgb


In [3]:
# read BPIC12.csv and Add label
# label = {'Activity' : 'O_ACCEPTED-COMPLETE'}

df = pd.DataFrame()

def label(self):
    df = pd.read_csv('BPIC12.csv')
    df = df[['Case ID', 'Activity', 'Resource', 'Complete Timestamp']]
    df['label'] = ""
    df.loc[df['Activity'] == 'O_ACCEPTED-COMPLETE', 'label'] = '1'
    df.loc[df['Activity'] != 'O_ACCEPTED-COMPLETE', 'label'] = '0'
    return df

df = label(df)

In [4]:
# error check
count = len(df[df['label'] == '1'])
print(count)

2243


In [5]:
# dict : drop_act, bucketing, encoding, model

# drop_act = {2, 4, 6, 8}
# bucketing = {1, 2*mean_trace_length}
# encoding = {'aggregate', 'index'}
# model = {'DT', 'RF', 'XGB', 'LGBM'}

parameter = {'drop_act':[2,4,6,8], 
    'bucketing':[1, 2*'mean_trace_length'],
    'encoding':['aggregate', 'index'],
    'model':['DT', 'RF', 'XGB', 'LGBM']}

     # need to modify bucketing

In [6]:
def random_machine():
    i = np.random.randint(0, 3)
    j = np.random.randint(0, 1)
    k = np.random.randint(0, 1)
    v = np.random.randint(0, 3)

    tuple = (parameter['drop_act'][i],
            parameter['bucketing'][j],
            parameter['encoding'][k],
            parameter['model'][v])

    return tuple

print(random_machine())

(2, 1, 'aggregate', 'RF')


In [32]:
rand = random_machine()
print(rand)

# ['DT', 'RF', 'XGB', 'LGBM']

if 'DT' in rand:
    DT()

if 'RF' in rand:
    RF()

if 'XGB' in rand:
    XGB()

if 'LGBM' in rand:
    LGBM()

(6, 1, 'aggregate', 'RF')
Accuracy: 1.0


In [18]:
# class genetic_algorithm()

In [19]:
# Data pre-processing

X = df.drop('label', axis=1)
y = df['label']

In [20]:
# one-hot encoding
# train and test set for decision tree
def train_set(self):
    df_train = pd.get_dummies(data = X, columns = ['Activity'], prefix = 'Activity')
    df_train['Complete Timestamp'] = pd.to_datetime(df_train['Complete Timestamp'], format='%Y/%m/%d %H:%M:%S.%f')

    # datetime to float for DT
    df_train['Complete Timestamp'] = (df_train['Complete Timestamp'] - df_train['Complete Timestamp'].min()) / np.timedelta64(1,'D')
    df_train

    # remove NaN for DT
    df_train = df_train.fillna(0)
    return df_train

a = train_set(X)
df_train = pd.DataFrame(a)

In [21]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=2023)

In [22]:
# machine learning
# decision tree
def DT():
    clf_dt = tree.DecisionTreeClassifier()
    clf_dt = clf_dt.fit(X_train, y_train)

    # Make predictions on the test data
    predictions = clf_dt.predict(X_test)

    # Evaluate the model's performance
    accuracy = clf_dt.score(X_test, y_test)
    print('Accuracy:', accuracy)

# random forest
def RF(): # kernel crush was that the RAM is too small.
    # Create a Random Forest classifier
    clf_rf = RandomForestClassifier()

    # Train the classifier on the training data
    clf_rf.fit(X_train, y_train)

    # Make predictions on the test data
    predictions = clf_rf.predict(X_test)

    # Evaluate the model's performance
    accuracy = clf_rf.score(X_test, y_test)
    print('Accuracy:', accuracy)

# XGB
# Encode the target variable
    
le = LabelEncoder()
y = le.fit_transform(y)

def XGB():
    xgb = XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=4)
    X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=2023)
    xgb.fit(X_train, y_train)
    xgb_pred = xgb.predict(X_test)

    accuracy = accuracy_score(y_test, xgb_pred)
    precision = precision_score(y_test, xgb_pred)
    recall = recall_score(y_test, xgb_pred)
    f1 = f1_score(y_test, xgb_pred)

    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1 Score:', f1)

# LGBM

def LGBM():
    lgb_clf = lgb.LGBMClassifier(num_leaves=31, objective='binary')
    lgb_clf.fit(X_train, y_train)
    y_pred = lgb_clf.predict(X_test)
    acc_lgbm = accuracy_score(y_test, y_pred)
    print('Accuracy', acc_lgbm)